# Notebook 3 : Analyse pour la Construction de la Couche Gold

## Objectif

Ce notebook vous guide dans l'**analyse des données de la couche silver** pour préparer la construction de la couche **gold** (analytics/BI/ML).

La couche gold contient des données transformées, nettoyées et optimisées pour l'analyse métier et la création de rapports/dashboards.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Exécuté le notebook `2_[LOAD]_load_to_bigquery.ipynb`** pour avoir toutes les tables dans BigQuery (dataset `silver`)
2. **Fichier `.env` configuré** avec les variables d'environnement nécessaires
3. **Service Account** avec les permissions BigQuery (`BigQuery Data Viewer`, `BigQuery Job User`)
4. **Packages Python installés** : `google-cloud-bigquery`, `pandas`, etc.

## Structure du Notebook

Ce notebook contient **4 tâches principales** à réaliser :

1. **Tâche 1 : Analyser la Granularité** - Comprendre le niveau de détail de chaque table
2. **Tâche 2 : Identifier les Transformations** - Déterminer les transformations nécessaires pour la couche gold
3. **Tâche 3 : Identifier les Clés de Jointure** - Mapper les relations entre les tables
4. **Tâche 4 : Analyse métier Identifier les **KPIs** métiers - 

## Résultats Attendus

À la fin de ce notebook, vous devriez avoir :
- Une compréhension claire de la structure et de la granularité de chaque table
- Une liste des transformations à appliquer pour créer la couche gold
- Un schéma de jointures documenté

## Configuration et Connexion à BigQuery

Cette section configure l'environnement et établit la connexion avec BigQuery pour explorer les données de la couche silver.


In [1]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

# Configuration
load_dotenv()

ROOT = Path.cwd().parent
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DATASET_ID = "silver"

# Authentification
creds = service_account.Credentials.from_service_account_file(SA_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print(f"[OK] - Connecté au projet: {PROJECT_ID}")
print(f"[OK] - Dataset: {DATASET_ID}")


[OK] - Connecté au projet: projectbigdataadama
[OK] - Dataset: silver


---

## Tâche 1 : Analyser la Granularité de Chaque Table

### Objectif

La **granularité** d'une table correspond au niveau de détail des données qu'elle contient. Comprendre la granularité est essentiel pour :
- Déterminer comment agréger les données
- Identifier les duplications potentielles
- Comprendre le niveau de détail nécessaire pour les analyses métier

### Instructions

Pour chaque table du dataset `silver`, vous devez :

1. **Lister les colonnes** et leurs types
2. **Identifier les clés primaires** ou les colonnes qui identifient de manière unique une ligne
3. **Déterminer la granularité** : à quel niveau de détail sont les données ?
   - Exemple : `fact_validations` pourrait être au niveau **jour × gare × type de titre**
4. **Compter les lignes** et estimer la taille des données
5. **Identifier les colonnes de dimension** (références vers d'autres tables)

### Exemple de Format de Réponse

```
Table: dim_gare
- Granularité: 1 ligne = 1 gare
- Clé primaire: id_gares
- Nombre de lignes: 1234
- Colonnes de dimension: aucune (table de dimension)
```


### À Compléter : Analyse de Granularité

**Tables de Dimension :**

1. `dim_gare`
2. `dim_ligne`
3. `dim_arret`
4. `dim_vacances_scolaires`
5. `dim_transporteur`

**Tables de Fait :**

6. `fact_validations_*` (toutes les tables de validation)

**Votre tâche :** Exécutez des requêtes SQL pour analyser chaque table et remplir le tableau ci-dessous.


In [10]:
tables_dimension = ["gares", "dim_ligne", "dim_arret", "dim_vacances_scolaires", "dim_transporteur"]

# Lister toutes les tables de fait dynamiquement
tables_fait = [t.table_id for t in bq_client.list_tables("silver_dataset") if t.table_id.startswith("fact_validations")]

# Fonction d'analyse
def analyse_table(table_name):
    print(f"\n=== Analyse de {table_name} ===")
    
    query = f"""
    SELECT *
    FROM `{PROJECT_ID}.silver_dataset.{table_name}`
    LIMIT 10
    """
    df = bq_client.query(query).to_dataframe()
    display(df)
    
    table = bq_client.get_table(f"{PROJECT_ID}.silver_dataset.{table_name}")
    print("Schéma :")
    for field in table.schema:
        print(f"  - {field.name}: {field.field_type} ({field.mode})")

# Analyser tables de dimension
for t in tables_dimension:
    analyse_table(t)

# Analyser tables de fait
for t in tables_fait:
    analyse_table(t)



=== Analyse de gares ===


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,geo_point_2d,geo_shape,id_gares,nom_gares,nom_so_gar,nom_su_gar,id_ref_zdc,nom_zdc,id_ref_zda,nom_zda,...,termetro,tertram,terval,exploitant,idf,principal,x,y,picto,nom_iv
0,POINT(2.37210313840257 48.9637554296203),POINT(2.37210313840257 48.9637554296203),643,Pierrefitte Stains,None,None,72787,Pierrefitte - Stains,43179,Pierrefitte - Stains,...,0,0,0,SNCF,1,0,654024.0938,6.873939e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Pierrefitte Stains
1,POINT(2.39059365264184 48.9767478482939),POINT(2.39059365264184 48.9767478482939),322,Garges-Sarcelles,None,None,65582,Garges - Sarcelles,43251,Garges - Sarcelles,...,0,0,0,SNCF,1,0,655389.1309,6.875373e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Garges-Sarcelles
2,POINT(2.4624564927797 48.6055558263784),POINT(2.4624564927797 48.6055558263784),270,Essonnes-Robinson,None,None,422776,Essonnes Robinson,45759,Essonnes Robinson,...,0,0,0,SNCF,1,0,660368.7153,6.834063e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Essonnes-Robinson
3,POINT(2.50175806584646 49.0493256008857),POINT(2.50175806584646 49.0493256008857),482,Louvres,None,None,69300,Louvres,44628,Louvres,...,0,0,0,SNCF,1,0,663577.5993,6.883387e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Louvres
4,POINT(2.35038953788977 48.9175671765846),POINT(2.35038953788977 48.9175671765846),806,Stade de France-Saint-Denis,None,None,72206,Stade de France Saint-Denis,43204,Stade de France Saint-Denis,...,0,0,0,SNCF,1,0,652391.9594,6.868816e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Stade de France-Saint-Denis
5,POINT(2.48424255643507 48.7066109356985),POINT(2.48424255643507 48.7066109356985),924,Yerres,None,None,63044,Yerres,43226,Yerres,...,0,0,0,SNCF,1,0,662048.2876,6.845287e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Yerres
6,POINT(2.46155018089749 48.5928121903408),POINT(2.46155018089749 48.5928121903408),894,Villabé,None,None,60124,Villabé,47904,Villabé,...,0,0,0,SNCF,1,0,660292.2364,6.832647e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Villabé
7,POINT(2.39678350341544 48.65484933543),POINT(2.39678350341544 48.65484933543),337,Grigny-Centre,None,None,60797,Grigny Centre,43132,Grigny Centre,...,0,0,0,SNCF,1,0,655568.8183,6.839578e+06,"{""thumbnail"": true, ""filename"": ""RER_D.svg"", ""...",Grigny-Centre
8,POINT(2.36127055524106 49.0984002624769),POINT(2.36127055524106 49.0984002624769),63,Belloy-Saint-Martin,None,None,67126,Belloy - Saint-Martin,47524,Belloy - Saint-Martin,...,0,0,0,SNCF,1,0,653352.0527,6.888919e+06,"{""thumbnail"": true, ""filename"": ""train_H.svg"",...",Belloy-Saint-Martin
9,POINT(2.33682968983683 49.0329720467715),POINT(2.33682968983683 49.0329720467715),239,Domont,None,None,66654,Domont,43120,Domont,...,0,0,0,SNCF,1,0,651506.0056,6.881657e+06,"{""thumbnail"": true, ""filename"": ""train_H.svg"",...",Domont


Schéma :
  - geo_point_2d: GEOGRAPHY (NULLABLE)
  - geo_shape: GEOGRAPHY (NULLABLE)
  - id_gares: INTEGER (REQUIRED)
  - nom_gares: STRING (NULLABLE)
  - nom_so_gar: STRING (NULLABLE)
  - nom_su_gar: STRING (NULLABLE)
  - id_ref_zdc: INTEGER (NULLABLE)
  - nom_zdc: STRING (NULLABLE)
  - id_ref_zda: INTEGER (NULLABLE)
  - nom_zda: STRING (NULLABLE)
  - idrefliga: STRING (NULLABLE)
  - idrefligc: STRING (NULLABLE)
  - res_com: STRING (NULLABLE)
  - indice_lig: STRING (NULLABLE)
  - mode: STRING (NULLABLE)
  - tertrain: STRING (NULLABLE)
  - terrer: STRING (NULLABLE)
  - termetro: STRING (NULLABLE)
  - tertram: STRING (NULLABLE)
  - terval: STRING (NULLABLE)
  - exploitant: STRING (NULLABLE)
  - idf: INTEGER (NULLABLE)
  - principal: INTEGER (NULLABLE)
  - x: FLOAT (NULLABLE)
  - y: FLOAT (NULLABLE)
  - picto: STRING (NULLABLE)
  - nom_iv: STRING (NULLABLE)

=== Analyse de dim_ligne ===


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


...

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,NB_VALD
0,2023-07-01,100,<NA>,<NA>,Inconnu,999999,Amethyste,57
1,2023-07-02,100,<NA>,<NA>,Inconnu,999999,Amethyste,43
2,2023-07-03,100,<NA>,<NA>,Inconnu,999999,Amethyste,57
3,2023-07-04,100,<NA>,<NA>,Inconnu,999999,Amethyste,64
4,2023-07-05,100,<NA>,<NA>,Inconnu,999999,Amethyste,28
5,2023-07-06,100,<NA>,<NA>,Inconnu,999999,Amethyste,4
6,2023-07-15,100,<NA>,<NA>,Inconnu,999999,Amethyste,1
7,2023-07-17,100,<NA>,<NA>,Inconnu,999999,Amethyste,2
8,2023-07-18,100,<NA>,<NA>,Inconnu,999999,Amethyste,1
9,2023-07-19,100,<NA>,<NA>,Inconnu,999999,Amethyste,1


Schéma :
  - JOUR: DATE (NULLABLE)
  - CODE_STIF_TRNS: INTEGER (NULLABLE)
  - CODE_STIF_RES: INTEGER (NULLABLE)
  - CODE_STIF_ARRET: INTEGER (NULLABLE)
  - LIBELLE_ARRET: STRING (NULLABLE)
  - ID_ZDC: INTEGER (NULLABLE)
  - CATEGORIE_TITRE: STRING (NULLABLE)
  - NB_VALD: INTEGER (NULLABLE)

=== Analyse de fact_validations_2024_s1_nb_fer_txt ===


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,NB_VALD
0,2024-01-07,100,<NA>,<NA>,Inconnu,999999,Amethyste,14
1,2024-01-08,100,<NA>,<NA>,Inconnu,999999,Amethyste,19
2,2024-01-09,100,<NA>,<NA>,Inconnu,999999,Amethyste,3
3,2024-01-16,100,<NA>,<NA>,Inconnu,999999,Amethyste,1
4,2024-01-30,100,<NA>,<NA>,Inconnu,999999,Amethyste,4
5,2024-01-31,100,<NA>,<NA>,Inconnu,999999,Amethyste,25
6,2024-02-22,100,<NA>,<NA>,Inconnu,999999,Amethyste,3
7,2024-03-12,100,<NA>,<NA>,Inconnu,999999,Amethyste,7
8,2024-03-13,100,<NA>,<NA>,Inconnu,999999,Amethyste,12
9,2024-03-14,100,<NA>,<NA>,Inconnu,999999,Amethyste,23


Schéma :
  - JOUR: DATE (NULLABLE)
  - CODE_STIF_TRNS: INTEGER (NULLABLE)
  - CODE_STIF_RES: INTEGER (NULLABLE)
  - CODE_STIF_ARRET: INTEGER (NULLABLE)
  - LIBELLE_ARRET: STRING (NULLABLE)
  - ID_ZDC: INTEGER (NULLABLE)
  - CATEGORIE_TITRE: STRING (NULLABLE)
  - NB_VALD: INTEGER (NULLABLE)


---

## Tâche 2 : Identifier les Transformations Nécessaires

### Objectif

Identifier les **transformations** à appliquer aux données de la couche silver pour créer la couche gold optimisée pour l'analyse.

### Types de Transformations Possibles

1. **Nettoyage des données**
   - Suppression des doublons
   - Gestion des valeurs NULL
   - Normalisation des formats (dates, textes)

2. **Enrichissement**
   - Ajout de colonnes calculées
   - Jointures avec les tables de dimension
   - Ajout de catégories/segments

3. **Agrégation**
   - Regroupement par dimensions (jour, gare, ligne, etc.)
   - Calcul de métriques (somme, moyenne, comptage)
   - Création de tables pré-agrégées

### Instructions

Pour chaque table, identifiez :
1. **Les problèmes de qualité** à corriger
2. **Les transformations nécessaires** avec des exemples concrets
3. **Les colonnes à ajouter** (calculées ou issues de jointures)
4. **Les agrégations possibles** pour optimiser les requêtes


In [ ]:
###Identification des colonnes de fact_validations
table_name = "fact_validations_2024_s1_nb_fer_txt"

table = bq_client.get_table(f"{PROJECT_ID}.silver_dataset.{table_name}")
print(f"Colonnes de {table_name} :")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type}")


Colonnes de fact_validations_2024_s1_nb_fer_txt :
  - JOUR: DATE
  - CODE_STIF_TRNS: INTEGER
  - CODE_STIF_RES: INTEGER
  - CODE_STIF_ARRET: INTEGER
  - LIBELLE_ARRET: STRING
  - ID_ZDC: INTEGER
  - CATEGORIE_TITRE: STRING
  - NB_VALD: INTEGER


In [ ]:
#### Détection des doublons
query = f"""
SELECT 
    JOUR, CODE_STIF_TRNS, CODE_STIF_RES, CODE_STIF_ARRET, 
    LIBELLE_ARRET, ID_ZDC, CATEGORIE_TITRE,
    COUNT(*) AS count_duplicates
FROM `{PROJECT_ID}.silver_dataset.{table_name}`
GROUP BY 
    JOUR, CODE_STIF_TRNS, CODE_STIF_RES, CODE_STIF_ARRET, 
    LIBELLE_ARRET, ID_ZDC, CATEGORIE_TITRE
HAVING COUNT(*) > 1
ORDER BY count_duplicates DESC
"""
df_duplicates = bq_client.query(query).to_dataframe()
print(f"=== Recherche de doublons dans {table_name} ===")
display(df_duplicates)


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Recherche de doublons dans fact_validations_2024_s1_nb_fer_txt ===


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,count_duplicates


In [19]:
print("Nombre de doublons détectés :", len(df_duplicates))


Nombre de doublons détectés : 0


In [21]:
import pandas as pd

tables_fait = [t.table_id for t in bq_client.list_tables("silver_dataset") if t.table_id.startswith("fact_validations")]

null_summary = []

for table_name in tables_fait:
    table = bq_client.get_table(f"{PROJECT_ID}.silver_dataset.{table_name}")
    columns = [field.name for field in table.schema]
    
    for col in columns:
        query = f"""
        SELECT COUNT(*) AS nb_nulls
        FROM `{PROJECT_ID}.silver_dataset.{table_name}`
        WHERE {col} IS NULL
        """
        df_null = bq_client.query(query).to_dataframe()
        null_summary.append({
            'Table': table_name,
            'Colonne': col,
            'Nombre_NULL': df_null['nb_nulls'][0]
        })

# Transformer en DataFrame unique
df_null_summary = pd.DataFrame(null_summary)
display(df_null_summary)



c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Table,Colonne,Nombre_NULL
0,fact_validations_2015s1_nb_fer_csv,JOUR,0
1,fact_validations_2015s1_nb_fer_csv,CODE_STIF_TRNS,0
2,fact_validations_2015s1_nb_fer_csv,CODE_STIF_RES,0
3,fact_validations_2015s1_nb_fer_csv,CODE_STIF_ARRET,0
4,fact_validations_2015s1_nb_fer_csv,LIBELLE_ARRET,0
...,...,...,...
131,fact_validations_2024_s1_nb_fer_txt,CODE_STIF_ARRET,905
132,fact_validations_2024_s1_nb_fer_txt,LIBELLE_ARRET,0
133,fact_validations_2024_s1_nb_fer_txt,ID_ZDC,0
134,fact_validations_2024_s1_nb_fer_txt,CATEGORIE_TITRE,0


In [23]:
import pandas as pd

# Liste des tables de fait
tables_fait = [t.table_id for t in bq_client.list_tables("silver_dataset") 
               if t.table_id.startswith("fact_validations")]

for table_name in tables_fait:
    print(f"\n=== Nettoyage des valeurs NULL dans {table_name} ===")
    
    # 1️⃣ Charger la table complète
    query = f"SELECT * FROM `{PROJECT_ID}.silver_dataset.{table_name}`"
    df = bq_client.query(query).to_dataframe()
    print(f"Nombre de lignes avant nettoyage : {len(df)}")
    
    # 2️⃣ Identifier les colonnes ayant au moins 1 NULL
    null_counts = df.isna().sum()
    cols_with_null = null_counts[null_counts > 0].index.tolist()
    print(f"Colonnes avec NULL : {cols_with_null}")
    
    if not cols_with_null:
        print("Pas de NULL à traiter dans cette table.")
        continue  # passer à la table suivante
    
    # 3️⃣ Colonnes critiques : suppression des lignes si NULL
    critical_cols = ['JOUR', 'CODE_STIF_TRNS', 'CODE_STIF_RES', 'CODE_STIF_ARRET']
    critical_with_null = [col for col in critical_cols if col in cols_with_null]
    if critical_with_null:
        df = df.dropna(subset=critical_with_null)
        print(f"Lignes supprimées pour colonnes critiques : {critical_with_null}")
    
    # 4️⃣ Colonnes non critiques : remplir les NULL
    non_critical_cols = [col for col in cols_with_null if col not in critical_cols]
    
    for col in non_critical_cols:
        if df[col].dtype == 'object':  # texte
            df[col] = df[col].fillna("Inconnu").str.strip().str.upper()
        elif 'int' in str(df[col].dtype) or 'float' in str(df[col].dtype):  # nombres
            df[col] = df[col].fillna(0)
    
    # 5️⃣ Aperçu des données nettoyées
    print(f"Nombre de lignes après nettoyage : {len(df)}")
    display(df.head())



=== Nettoyage des valeurs NULL dans fact_validations_2015s1_nb_fer_csv ===
Nombre de lignes avant nettoyage : 755989
Colonnes avec NULL : ['ID_REFA_LDA']
Nombre de lignes après nettoyage : 755989


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_REFA_LDA,CATEGORIE_TITRE,NB_VALD
0,2015-02-15,100,110,144,CHAMPS-ELYSEES-CLEMENCEAU,71305,NON DEFINI,17
1,2015-02-16,100,110,144,CHAMPS-ELYSEES-CLEMENCEAU,71305,NON DEFINI,23
2,2015-02-17,100,110,144,CHAMPS-ELYSEES-CLEMENCEAU,71305,NON DEFINI,21
3,2015-02-18,100,110,144,CHAMPS-ELYSEES-CLEMENCEAU,71305,NON DEFINI,24
4,2015-02-19,100,110,144,CHAMPS-ELYSEES-CLEMENCEAU,71305,NON DEFINI,24



=== Nettoyage des valeurs NULL dans fact_validations_2015s2_nb_fer_csv ===


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Nombre de lignes avant nettoyage : 778747
Colonnes avec NULL : ['ID_REFA_LDA']
Nombre de lignes après nettoyage : 778747


...

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,NB_VALD
84,2023-07-01,100,110,1,PORTE MAILLOT,71379,Amethyste,147
85,2023-07-02,100,110,1,PORTE MAILLOT,71379,Amethyste,86
86,2023-07-03,100,110,1,PORTE MAILLOT,71379,Amethyste,158
87,2023-07-04,100,110,1,PORTE MAILLOT,71379,Amethyste,184
88,2023-07-05,100,110,1,PORTE MAILLOT,71379,Amethyste,183



=== Nettoyage des valeurs NULL dans fact_validations_2024_s1_nb_fer_txt ===


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Nombre de lignes avant nettoyage : 859043
Colonnes avec NULL : ['CODE_STIF_RES', 'CODE_STIF_ARRET']
Lignes supprimées pour colonnes critiques : ['CODE_STIF_RES', 'CODE_STIF_ARRET']
Nombre de lignes après nettoyage : 858138


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,NB_VALD
105,2024-01-01,100,110,1,PORTE MAILLOT,71379,Amethyste,78
106,2024-01-02,100,110,1,PORTE MAILLOT,71379,Amethyste,133
107,2024-01-03,100,110,1,PORTE MAILLOT,71379,Amethyste,204
108,2024-01-04,100,110,1,PORTE MAILLOT,71379,Amethyste,177
109,2024-01-05,100,110,1,PORTE MAILLOT,71379,Amethyste,179


---

## Tâche 3 : Identifier les Clés de Jointure Possibles

### Objectif

Identifier toutes les **relations possibles** entre les tables pour pouvoir créer des jointures dans la couche gold.

### Instructions

Pour chaque paire de tables, identifiez :

1. **Les colonnes de jointure** (clés étrangères)
2. **Le type de relation** (1-1, 1-N, N-N)
3. **La cardinalité** (combien de lignes de la table A correspondent à combien de lignes de la table B)
4. **Vérifier l'intégrité référentielle** (toutes les clés étrangères existent-elles dans la table de dimension ?)


In [26]:
# Tables de dimension
tables_dim = ['gares', 'dim_ligne', 'dim_arret', 'dim_transporteur', 'dim_vacances_scolaires']

# Toutes les tables de fait
tables_fait = [t.table_id for t in bq_client.list_tables("silver_dataset") 
               if t.table_id.startswith("fact_validations")]

for fact_table in tables_fait:
    print(f"\n=== Analyse des jointures pour {fact_table} ===")
    
    fact_table_obj = bq_client.get_table(f"{PROJECT_ID}.silver_dataset.{fact_table}")
    fact_cols = [field.name for field in fact_table_obj.schema]
    
    for dim_table in tables_dim:
        dim_table_obj = bq_client.get_table(f"{PROJECT_ID}.silver_dataset.{dim_table}")
        dim_cols = [field.name for field in dim_table_obj.schema]
        
        common_cols = list(set(fact_cols) & set(dim_cols))
        if not common_cols:
            print(f"Aucune clé commune avec {dim_table}")
            continue
        
        for col in common_cols:
            print(f"\nTable de dimension : {dim_table} | Colonne commune : {col}")
            query_integrity = f"""
            SELECT COUNT(*) AS missing_keys
            FROM `{PROJECT_ID}.silver_dataset.{fact_table}` f
            LEFT JOIN `{PROJECT_ID}.silver_dataset.{dim_table}` d
            ON f.{col} = d.{col}
            WHERE d.{col} IS NULL
            """
            missing_keys = bq_client.query(query_integrity).to_dataframe()['missing_keys'][0]
            integrity = "OK" if missing_keys == 0 else f"{missing_keys} manquants"
            print(f"Intégrité référentielle : {integrity}")



=== Analyse des jointures pour fact_validations_2015s1_nb_fer_csv ===
Aucune clé commune avec gares
Aucune clé commune avec dim_ligne
Aucune clé commune avec dim_arret
Aucune clé commune avec dim_transporteur
Aucune clé commune avec dim_vacances_scolaires

=== Analyse des jointures pour fact_validations_2015s2_nb_fer_csv ===
Aucune clé commune avec gares
Aucune clé commune avec dim_ligne
Aucune clé commune avec dim_arret
Aucune clé commune avec dim_transporteur
Aucune clé commune avec dim_vacances_scolaires

=== Analyse des jointures pour fact_validations_2016s1_nb_fer_txt ===
Aucune clé commune avec gares
Aucune clé commune avec dim_ligne
Aucune clé commune avec dim_arret
Aucune clé commune avec dim_transporteur
Aucune clé commune avec dim_vacances_scolaires

=== Analyse des jointures pour fact_validations_2016s2_nb_fer_txt ===
Aucune clé commune avec gares
Aucune clé commune avec dim_ligne
Aucune clé commune avec dim_arret
Aucune clé commune avec dim_transporteur
Aucune clé commune 

---

## Tâche 4 : Analyse métier

Identifier les **KPIs** (Key Performance Indicators) à calculer :
- Nombre total de validations par période
- Répartition par type de titre
- Top 10 des gares les plus fréquentées
- Comparaison jour ouvrable vs weekend



In [ ]:
#1️⃣ Nombre total de validations par jour
fact_table = "fact_validations_2024_s1_nb_fer_txt"

query_total = f"""
SELECT 
    JOUR, 
    SUM(NB_VALD) AS total_validations
FROM `{PROJECT_ID}.silver_dataset.{fact_table}`
GROUP BY JOUR
ORDER BY JOUR
"""

df_total = bq_client.query(query_total).to_dataframe()
print("=== Nombre total de validations par jour ===")
display(df_total)


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Nombre total de validations par jour ===


,JOUR,total_validations
0,2024-01-01,1314385
1,2024-01-02,4091678
2,2024-01-03,4807974
3,2024-01-04,5091641
4,2024-01-05,4935250
...,...,...
177,2024-06-26,6288567
178,2024-06-27,6398909
179,2024-06-28,6055079
180,2024-06-29,4227125


In [32]:
#2️⃣ Répartition par type de titre
query_titre = f"""
SELECT 
    CATEGORIE_TITRE, 
    SUM(NB_VALD) AS total_validations
FROM `{PROJECT_ID}.silver_dataset.{fact_table}`
GROUP BY CATEGORIE_TITRE
ORDER BY total_validations DESC
"""

df_titre = bq_client.query(query_titre).to_dataframe()
print("=== Répartition des validations par type de titre ===")
display(df_titre)


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Répartition des validations par type de titre ===


,CATEGORIE_TITRE,total_validations
0,Forfait Navigo,492335516
1,Imagine R,198765739
2,Contrat Solidarité Transport,120948494
3,Autres titres,82016303
4,NON DEFINI,26054833
5,Amethyste,20795674
6,Forfaits courts,7377389


In [ ]:
#3️⃣ Top 10 des gares les plus fréquentées
query_top_gares = f"""
SELECT 
    LIBELLE_ARRET AS gare, 
    SUM(NB_VALD) AS total_validations
FROM `{PROJECT_ID}.silver_dataset.{fact_table}`
GROUP BY gare
ORDER BY total_validations DESC
LIMIT 10
"""

df_top_gares = bq_client.query(query_top_gares).to_dataframe()
print("=== Top 10 des gares les plus fréquentées ===")
display(df_top_gares)


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Top 10 des gares les plus fréquentées ===


,gare,total_validations
0,SAINT-LAZARE,34191074
1,CHATELET,22050179
2,GARE DE LYON,19696028
3,LA DEFENSE,16076541
4,GARE DU NORD,15270456
5,MONTPARNASSE,14823464
6,GARE DE L'EST,13419660
7,LES HALLES,7587515
8,REPUBLIQUE,7421493
9,NATION,6237082


In [36]:
#4️⃣ Comparaison jour ouvrable vs weekend
query_jour_weekend = f"""
SELECT
    CASE 
        WHEN EXTRACT(DAYOFWEEK FROM JOUR) IN (1,7) THEN 'Weekend'
        ELSE 'Jour ouvrable'
    END AS type_jour,
    SUM(NB_VALD) AS total_validations
FROM `{PROJECT_ID}.silver_dataset.{fact_table}`
GROUP BY type_jour
ORDER BY type_jour
"""

df_jour_weekend = bq_client.query(query_jour_weekend).to_dataframe()
print("=== Comparaison Jour ouvrable vs Weekend ===")
display(df_jour_weekend)


c:\Users\etudiant\AppData\Local\Programs\Python\Python314\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


=== Comparaison Jour ouvrable vs Weekend ===


,type_jour,total_validations
0,Jour ouvrable,770488340
1,Weekend,177805608
